In [31]:
import pandas as pd
import jieba
from torchtext.vocab import FastText
import torch
import numpy as np

In [32]:
#是否使用gpu
use_cuda=False

In [33]:
def get_text(x):
    record=x["result"]
    t=''.join([str(i["type"]) for i in record])
    detail=''.join([str(i["detail"]) for i in record])
    other=''.join([str(i["other"]) for i in record])
    return "".join([t,detail,other])

In [34]:
class fasttext(torch.nn.Module):
    def __init__(self):
        super(fasttext,self).__init__()
        self.net=torch.nn.Sequential(
        torch.nn.Linear(300,256),
        torch.nn.ReLU(),
        torch.nn.Linear(256,32),
        torch.nn.ReLU(),
        torch.nn.Linear(32,2),
        torch.nn.Softmax(dim=1))
    
    def forward(self,x):
        out=self.net(x)
        return out

In [35]:
def bigram(x):
    vectors=FastText(language="zh")
    xx=[]
    for i in range(len(x)-1):
        xx+=[x[i]+x[i+1]]
    if use_cuda:
        return torch.mean(torch.stack(list(map(lambda y:vectors[y],xx))),dim=0).to("cuda")
    else:
        return torch.mean(torch.stack(list(map(lambda y:vectors[y],xx))),dim=0).to("cuda")

In [36]:
#vscode csv to json插件转换的
data_n=pd.read_json('/home/yuanmanjie/che300/negative_500.json',orient='records')
data_p=pd.read_json('/home/yuanmanjie/che300/positive_500.json',orient='records')

In [37]:
data_n["data"]=data_n.iloc[:,1].map(get_text)
data_p["data"]=data_p.iloc[:,1].map(get_text)

In [38]:
data=pd.concat([data_n,data_p])
data["embed"]=data["data"].map(bigram)
data=data.sample(frac=1)

In [39]:
#保存一下避免之后重跑上面的代码 很耗时……
data.to_excel(r"processed_data.xlsx",index=0)
data=pd.read_excel(r"processed_data.xlsx")

In [40]:
if use_cuda:
    train_x=torch.stack(list(data["embed"]),dim=0).to("cuda")
    train_y=torch.tensor(list(data["label"])).long().to("cuda")
else:
    train_x=torch.stack(list(data["embed"]),dim=0)
    train_y=torch.tensor(list(data["label"])).long()    
#ones=torch.sparse.torch.eye(2)
#train_y=ones.index_select(0,train_y).long()

TypeError: expected Tensor as element 0 in argument 0, but got str

In [ ]:
if use_cuda:
    loss=torch.nn.CrossEntropyLoss(weight=torch.tensor([1.,5.]).to("cuda"))
else:
    loss=torch.nn.CrossEntropyLoss(weight=torch.tensor([1.,5.])
#loss=torch.nn.MSELoss()
epoch_n=1000
learning_rate=1e-3


In [ ]:
if use_cuda:
    model=fasttext().to("cuda")
else:
    model=fasttext()
optimzer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [ ]:
ttrain_x=train_x[:800]
ttrain_y=train_y[:800]
pre_x=train_x[800:]
pre_y=train_y[800:]

In [ ]:
def test(y,pred):
    l=loss(pred,y)
    print("Loss:{:.4f}".format(l.data),end='')
    ly=pred[:,0]<pred[:,1]
    TP=int(((y==1)&(ly==1)).sum())
    TN=int(((y==0)&(ly==0)).sum())
    FN=int(((y==1)&(ly==0)).sum())
    FP=int(((y==0)&(ly==1)).sum())
    if TP+FN!=0:
        r=TP/(TP+FN)
    else:
        r=-1
    if TP+FP!=0:
        p=TP/(TP+FP)
    else:
        p=-1
    print("p:{:.4f}".format(p),end='')
    print("r:{:.4f}".format(r),end='')
    if r>=0 and p >=0:
        print("acc:{:.4f}".format((TP+TN)/(TP+FP+FN+TN)),end='')
    if r!=0 or p!=0:
        print("F1:{:.4f}".format(2*r*p/(r+p)),end='')
    print("")

In [ ]:
def te(m,mode=0):
    if mode==0:
        pred=m(pre_x)
        test(pre_y,pred)
    else:
        pred=m(ttrain_x)
        test(ttrain_y,pred)

In [43]:
for epoch in range(epoch_n):
    y_pred=model(ttrain_x)
    #print(y_pred)
    l=loss(y_pred,ttrain_y)
    if epoch%100==0:
        print("Epoch:{},Loss:{:.4f}".format(epoch,l.data))
#     if epoch%100==0:
        print("training set:",end='')
        te(model,1)
        print("test set:",end='')
        te(model,0)
        
    optimzer.zero_grad()
    l.backward()
    #for param in model.parameters():
    #    param.data-=param.grad.data*lr
    optimzer.step()

Epoch:0,Loss:0.3876
training set:Loss:0.3876p:0.7219r:0.9899acc:0.8063F1:0.8349
test set:Loss:0.4143p:0.6993r:0.9615acc:0.7650F1:0.8097
Epoch:100,Loss:0.3861
training set:Loss:0.3861p:0.7273r:0.9899acc:0.8113F1:0.8385
test set:Loss:0.4145p:0.7042r:0.9615acc:0.7700F1:0.8130
Epoch:200,Loss:0.3848
training set:Loss:0.3848p:0.7300r:0.9899acc:0.8137F1:0.8403
test set:Loss:0.4148p:0.6993r:0.9615acc:0.7650F1:0.8097
Epoch:300,Loss:0.3835
training set:Loss:0.3835p:0.7382r:0.9899acc:0.8213F1:0.8457
test set:Loss:0.4152p:0.6993r:0.9615acc:0.7650F1:0.8097
Epoch:400,Loss:0.3823
training set:Loss:0.3823p:0.7410r:0.9899acc:0.8237F1:0.8476
test set:Loss:0.4162p:0.6972r:0.9519acc:0.7600F1:0.8049
Epoch:500,Loss:0.3814
training set:Loss:0.3814p:0.7438r:0.9899acc:0.8263F1:0.8494
test set:Loss:0.4171p:0.6972r:0.9519acc:0.7600F1:0.8049
Epoch:600,Loss:0.3807
training set:Loss:0.3807p:0.7438r:0.9899acc:0.8263F1:0.8494
test set:Loss:0.4181p:0.7021r:0.9519acc:0.7650F1:0.8082
Epoch:700,Loss:0.3800
training set:L

Loss:0.5436
p:0.7387
r:0.8039
acc:0.7550
F1:0.7700


In [41]:
data

,callback_content,label,vin,data,embed
113,"{'last_time_to_shop': '2017-12-18', 'total_mil...",1,070ffb476dbb3c1fe\r,小修保险事故保养保养保险保养索赔保养首保PDI检查空调后备箱盖拆装;后备箱盖喷漆;后尾板补漆...,"tensor([-1.5412e-02, 7.4362e-02, -6.1587e-02,..."
17,"{'last_time_to_shop': '2019-02-25', 'total_mil...",0,00f577f441efcb8b6\r,保养免保保养首次保养蓝壳合成技术SN级别发动机油5W-40（4L）;保养;蓝壳合成技术SN级...,"tensor([-2.2128e-02, 1.3787e-01, -1.0860e-01,..."
188,"{'last_time_to_shop': '2018-07-20', 'total_mil...",0,0b76778ebc0b60f92\r,预约预约索赔免费保养九大安全检测(价值128元);发动机深化保养;节流阀清洗;更换机油机滤;...,"tensor([-3.9833e-03, 2.0603e-01, -1.2353e-01,..."
384,"{'last_time_to_shop': '2016-06-04', 'total_mil...",0,15e6b5b68dd44eb6f\r,一般：客户付款定期保养定期保养定期保养喷漆：客户付款定期保养定期保养定期保养定期保养一般：客...,"tensor([-0.0330, 0.1882, -0.1762, 0.0545, 0..."
475,"{'last_time_to_shop': '2017-05-12', 'total_mil...",1,1b01e918743c20dc0\r,索赔钣金喷漆免费保养2免费保养1更换起动机; 客户反映车辆无法启动，需救援更换后排气管;更换...,"tensor([-2.3054e-02, 1.2849e-01, -9.9300e-02,..."
16,"{'last_time_to_shop': '2017-05-27', 'total_mil...",1,00ca6060090efd075\r,保险免费检测小修保养保养市场服务首保索赔四轮定位;行李盖喷漆;右后叶子板喷漆;左后叶子板修复...,"tensor([-0.0106, 0.0419, -0.0360, 0.0145, 0..."
239,"{'last_time_to_shop': '2016-06-11', 'total_mil...",0,0e1d43d812599701b\r,其它其它其它其它其它机油保养其它其它其它其它其它其它其它其它其它其它其它其它其它其它其它其它...,"tensor([-1.9970e-02, 1.9356e-01, -1.5206e-01,..."
420,"{'last_time_to_shop': '2014-09-03', 'total_mil...",0,18b0c6b949a4568ef\r,一般维修一般维修一般维修一般维修一般维修保修保修免费+1服务活动发动机仓清洁;换机油两滤;更...,"tensor([-0.0170, 0.2299, -0.1497, 0.0675, 0..."
116,"{'last_time_to_shop': '2016-07-31', 'total_mil...",1,0719ed7167ee3a571\r,一般维修常规保养事故车维修常规保养常规保养常规保养常规保养质量担保保养事故车维修更换机油机滤...,"tensor([-3.7653e-02, 8.7964e-02, -8.6661e-02,..."
351,"{'last_time_to_shop': '2018-08-01', 'total_mil...",0,13c29a27e0b79e0a0\r,维修维修维修维修维修维修保养保养维修维修维修维修维修维修维修维修维修维修维修维修维修保养维修...,"tensor([-9.8629e-03, 1.1508e-01, -7.4287e-02,..."


tensor([-0.0448,  1.1315, -0.7133,  0.2258,  0.9335, -0.1075, -0.8293, -0.1014,
         0.3125, -0.1808,  0.8690,  0.1376,  1.9608, -1.1774,  0.2758, -0.7833,
         1.1592, -0.3257, -1.3767, -1.1352, -0.8417,  1.3421,  1.3970,  0.1438,
         1.0305, -0.4318,  1.2344,  0.2403,  0.7931,  0.3357,  1.4705, -1.1395,
        -1.0604, -0.7375, -1.1902,  1.2162,  1.1020, -0.5441,  0.4227,  1.5478,
         1.2682,  1.1972,  1.5960, -1.4509, -0.6763, -0.6861, -1.0299, -0.2203,
         0.8798, -1.0690,  1.4894, -0.7151, -0.5835, -0.5646,  0.2686, -1.0477,
        -1.1429, -1.5862, -4.3141,  0.9801,  1.1265, -0.1549,  0.9980,  1.2995,
         0.1297,  1.2300, -1.0817, -0.3750, -0.7593, -0.2319,  1.6325, -0.7496,
         1.4902, -0.2878, -1.1078,  0.6162, -0.1616, -1.5167, -0.4369, -1.1276,
        -0.2838,  0.4398,  1.2792, -0.8688, -0.9199, -0.8966,  0.2349,  0.8369,
         0.2281, -1.3591, -0.8649,  0.8512, -0.2739, -0.3199, -0.9880,  1.2700,
         0.7796,  0.4996,  0.0162,  0.19